In [ ]:
import joblib
import insightface
import numpy as np
from sklearn.preprocessing import LabelEncoder, Normalizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from data_loading import load_pictures, get_features
from tensorflow.keras.models import load_model

In [ ]:
PATH = 'your path to dataset'
load_pictures(PATH)

In [ ]:
retina_model = insightface.model_zoo.get_model('retinaface_r50_v1')
retina_model.prepare(ctx_id = -1, nms=0.4)
facenet_model = load_model('facenet_keras.h5')

In [ ]:
data = np.load('data.npz', allow_pickle=True)
get_features(data['arr_0'], data['arr_1'], retina_model, facenet_model)

In [ ]:
data = np.load('features.npz', allow_pickle=True)
x_train, x_test, y_train, y_test = train_test_split(data['arr_0'], data['arr_1'], test_size=0.2, random_state=42)

In [ ]:
in_encoder = Normalizer(norm='l2')
x_train = in_encoder.fit_transform(np.asarray(x_train))
x_test = in_encoder.transform(np.asarray(x_test))

In [ ]:
out_encoder = LabelEncoder()
y_train = out_encoder.fit_transform(y_train)
y_test = out_encoder.transform(y_test)

In [ ]:
model = SVC(kernel='linear', probability=True)
model.fit(x_train, y_train)

In [ ]:
yhat_train = model.predict(x_train)
yhat_test = model.predict(x_test)
score_train = accuracy_score(y_train, yhat_train)
score_test = accuracy_score(y_test, yhat_test)
print('Accuracy: train = {:.2%}, test = {:.2%}'.format(score_train, score_test))

In [ ]:
joblib.dump(in_encoder, 'in_encoder.joblib')
joblib.dump(out_encoder, 'out_encoder.joblib')
joblib.dump(model, 'model.joblib')